# Introduccion 

El presente documento contiene el informe con el trabajo realizado para la primra práctica de Computación Evolutiva por el alumno Andrés Mañas Mañas.

El lenguaje de programación elegido para la realización de la práctica es **Clojure** (un dialecto de Lisp). He elegido este lenguaje porque hace muy sencillo la representación de conocimiento como estructuras de datos muy fácilmente manipulables. Es además mi lenguaje favorito.

En el material entregado para esta práctica se incluye el archivo **README.pdf**. Es del todo recomendable leer tal archivo en primer lugar para:

  - documentarse sobre cómo disponer de un entorno en el que ejecutar el código fuente de la práctica
  - visualizar rápidamente la estructura de árbol con todos los documentos que se incluyen en la práctica  
  
Parte de los entrgables de la práctica consisten en un sistema que permita ver la evolución del algoritmo y los resultados alcanzdos. No son tantas las librerías que permiten visualizar gráficos en Clojure. Si acaso la más destacada sea [http://incanter.org/](http://incanter.org/). Sin embargo es una librería pesadísima, y he preferido no agregarla como dependencia del proyecto. Además de lo desagradable que es que al ejecutar los algoritmos se vayan abriendo popups en background.  

Por eso me he tomado la libertad de desplegar en un pequeño servidor que tengo en AWS un web service en php y un panel html que permite ver en tiempo real la evolución de una ejecución del algoritmo de aprendizaje.  

Cuando se lanza una ejecución de un experimento, de forma transparente se envían periódicamente estadísticas del avance del algoritmo a [http://amanas.ml/ce/service.php](http://amanas.ml/ce/service.php)  

Por otro lado, si simultáneamente tenemos abierta en un navegador la página [http://amanas.ml/ce/status.html](http://amanas.ml/ce/status.html), podrémos ir viendo en tiempo real y de forma gráfica la evolución del aprendizaje del experimento que estemos ejecutando en cada momento.  


## MUY IMPORTANTE !!!

Hay que disponer de conexión a internet y abrír [http://amanas.ml/ce/status.html](http://amanas.ml/ce/status.html) cuando ustedes vayan a probar los algoritmos que les entrego con estos materiales.

![Figura 1. Selector de experimentos](ai/ce/p1/doc/img/selector.png)

Cada vez que se inicia un experimento, uno de los parámetros que se indican en la configuración del mismo es el nombre del experimento. Después, en la página [http://amanas.ml/ce/status.html](http://amanas.ml/ce/status.html) basta seleccionar del combo box el nombre que se ha indicado en la configuración y así se puede ver el resultado de la ejecución del algoritmo. 

Estos informes gráficos tienen configurado un tiempo de vida en el servidor que debiera ser suficiente para que siguieran estando disponibles cuando ustedes decidan visualizarlos. En el caso de todos los realizados por mí, el nombre siempre lo empiezo por "amanas: ...". De este modo, si ustedes realizarn experimentos de los mismo ejemplos que yo he realizado, pueden comparar las distintas realizaciones.


# Descripción del problema y requisitos  

En la primera práctica se nos pide que implementedmos desde cero un algoritmo genético para la resolución del **Problema de la Mochila Binario**.  

El **Problema de la Mochila Binario** se describe así:  

>> Dada una mochila con cierta “capacidad” y varios objetos con cierto “volumen” y “valor”, se trata de determinar qué objetos hay que introducir en la mochila para maximizar el valor total de los objetos contenidos en la misma. Obviamente, se debe cumplir que la suma de los volúmenes de los objetos introducidos en la mochila no exceda la capacidad de ésta.  

Los requisito pedidos son:

  - representación binaria de los individuos
  - función fitness basada en ordenamiento por ratio de los objetos y de forma que trate de forma adecuada aquellos individuos que representan soluciones no factibles al problema de la mochila binario
  - población inicial aleatoria
  - selección de padres por torneo
  - cruce por punto con probabilidad dada
  - mutación de todos los genes con una probabilidad dada
  - selección de supervivientes atendiendo a un modelo generaciónal con elitismo
    


# Implementación

Como dije más arriba, el lenguaje que he seleccionado es **Clojure**, un dialecto de Lisp que se engloba por lo tanto en el paradigma de programación funcional.

A mi juicio, las ventajes de utilizar un lenguaje como Clojure son:

- facilidad en la representación del conocimiento, parte clave del problema
- sencillez en la programación de las distintas funciones clave en el algoritmo de aprendizaje. Es un lenguaje de alto nivel que permite realizar operaciones de mapeo o filtrado con una simple función, por lo que se ahora mucho código y por lo tanto se evita complejidad en la solución desde la base
- evitar los problemas relacionados con la concurrencia y la mutación del estado de los objetos en el paradigma de la orientación a objetos
- y por qué no, diversión garantizada pues Clojure es el lenguaje más divertido que conozco, y puestos a trabajar, mejor hacerlo con alegría

La única desventaja que le veo a la utilización de clojure es que no es el lenguaje que mejor rendimiento ofrece. Corre sobre la máquina virtual java (aunque también hay intérpretes para .Net o javascript). Y, aunque dependiendo del caso puede ofrecer rendimientos muy superiores a, por ejemplo, java, cuando la carga de computación es elevada suelo flaquear un poco. En este sentido, lenguajes como C seguro que brindarían rendimientos mejores.

La implementación del algoritmo que satisface las especificacione indicadas anteriormente pueden ustedes encontrarlas en el archivo **src/ce/p1.clj** 

Vamos a comentar un poco las partes claves del código que entrego.

## Representación del conocimiento

Tenemos que representar como mínimo dos conceptos clave: los objetos (genes) y los individuos.

**Cada objeto lo represento como un mapa** (estructura clave-valor) con tres claves: nombre del objeto, valor del objeto y volumen del objeto.


```clojure
{:nam "Objeto 0" :val 1 :vol 3}
```

Como parte del preprocesado inicial de los objetos de entrada del problema, agrego automáticamente una nueva propiedad a cada objeto que será el **ratio (valor/volumen)**, de modo que en los distintos accesos que haga al ojeto no tenga que recalcular este valor y así ahorro en computación. Es decir, quedaría:


```clojure
{:nam "Objeto 0" :val 1 :vol 3 :ratio 1/3}
```

Ésta sería la representación de cada objeto individualmente.

Pero además, en las primeras fases del algoritmo, lo que hago es **ordenar todos los objetos en un array por orden de ratio**. Es decir, los objetos con mayor valor y menor volumen los primeros y los de menor valor y mayor volumen los últimos. La razón de esta ordenación es mejorar el rendimiento de los cálculos que se realizarán para determinar el valor de fitness de cada individuo a lo largo de la la evolución.

De este modo, **el conjunto de objetos que tenemos** para meter en la mochila **acabaría representado** de un modo similar al siguiente:


```clojure
[{:nam "Objeto 9" :val 10 :vol 3 :ratio 10/3}
 {:nam "Objeto 5" :val 7  :vol 3 :ratio 7/3}
 {:nam "Objeto 2" :val 8  :vol 4 :ratio 2}
 ...]
```

Tenemos que determinar además cómo representar a los individuos. Siguiendo las indicaciones decido **representar cada individuo como un vector de genes binarios**. Es decir, si el gen n-ésimo de un individuo es `True`, con $0 <= n < \text{número de objetos}$, entonces ese individuo incluiría en la mochila el objeto n-ésimo. Y no lo incluiría si el gen ubiera tenido el valor `False`. Nótese que en lugar de [0, 1] utilizo [True, False], sencillamente porque se me antoja más claro.

Por ejemplo, el individuo [True False True...] representa meter en la mochila el primer y tercer objetos (una vez reordenados por ratio, según se comentó anteriormente).




## Funciones más relevantes

```clojure
;; Función que devuelve el valor de aptitud o conveniencia de un individuo.
;; Procedimiento:
;;  - se queda con los objetos que tiene en la mochila (gen true)
;;    ordenados por ratio descendente
;;  - acumula los valores de los objetos, mientras quepan en la mochila
;;  - devuelve el valor acumulado
;; Parámetros:
;;  - individual: el individuo
(defn fitness [individual]...

;; Devuelve la representación de un individuo como el 
;; conjunto de objetos que introduce en la mochila.
(defn decode [individual]...

;; Genera un objeto a partir de sus propiedades.
;; - nam: nombre del objeto
;; - val: valor del objeto
;; - vol: volumen del objeto
(defn new-object...

;; Genera un objeto aleatoriamente con valor entre 0 y max-val y
;; volumen entre 1 y max-vol.
;; - i: número de objeto
;; - max-val: valor máximo
;; - max-vol: volument máximo
(defn rand-object [i max-val max-vol]...
  
;; Genera aleatoriamente un individuo.
(defn rand-individual []...

;; Genera aleatoriamente una población.
(defn rand-population []...

;; Selecciona el primer elmento de una lista con probabilidad 
;; stochastic-prob. De no ser seleccionado, selecciona el 
;; primero del resto con probabilidad stochastic-prob también. 
;; Y así hasta agotar la lista.
(defn first-stochastic [col]...

;; Selecciona por torneo estocástico.
;;  - population: población de la que se selecciona
;;  - size: número de individuos a seleccionar
(defn tournament-stochastic [population size]...

;; Cruza dos padres por un punto o devuelve los 
;; padres tal cual, dependiendo de la probabilidad de cruze.
;;  - parent1: primer padre
;;  - parent2: segundo padre
(defn crossover-one-point [parent1 parent2]...

;; Muta los genes de un individuo atendiendo a una 
;; probabilidad de mutación data por 1/número de objetos del individuo.
;;  - individual: el individuo a mutar
(defn mutate [individual]...

;; Variable donde almacenamos los mejores individuos 
;; de la historia reciente.
;; Permite parar la evolución si no se mejora en un número dado de iteraciones
(def best-history (atom []))

;; Determina si se llevan demasiadas generaciones sin que aparezcan individuos
;; con fitness mejorado.
;;  - generation: la generación en curso
;;  - best: el mejor individuo de la generación
(defn enough-blockage? [generation best]...

;; Determina si la evolución ha llegado a su fin, bien por haberse alcanzado
;; demasiadas generaciones, bien por haberse alcanzado un individuo con
;; fitness suficiente, bien por llevar demasiadas generaciones sin que se
;; incremente el fitness.
;;  - generation: la generación en curso
;;  - best: el mejor individuo de la generación
(defn done? [generation best]...

;; Construye la nueva generación a partir de la generación actual
;; aplicando el modelo generacional.
;;  - population: la población actual
(defn build-offspring [population]...

;; Inicializa y lleva a cabo la evolución. Reporta el resultado.
;; Devuelve el mejor individuo encontrado.
;; - config: mapa con la configuración del experimento. Adopta esta forma:
;; {:pack-size 755
;;  :objects objects
;;  :population-size 50
;;  :tournament-size 5
;;  :replacement true
;;  :rand-gen-prob 5/10
;;  :stochastic-prob 8/10
;;  :crossover-prob 5/10
;;  :max-generations 200
;;  :idle-generations 20
;;  :report-delta 1
;;  :name "Nombre del experimento"}
(defn go-live [config]...
```


## Ejecución programática del algoritmo

Por último, es necesario indicar el modo de representación de los metaparámetros del algoritmo y la forma de iniciar la ejecución del mismo. 

En la implementación que entrego **todos los metaparámetros del algoritmo son configurables** en el momento de su lanzamiento. 

De este modo, la función que lleve a cabo la ejecución de un experimento es:


```clojure
(defn go-live [config]
   ...
```    

a la que se le pasa un mapa con los parámetros de configuración. Estos parámetros son los siguientes:  


```clojure
 {;; El tamaño de la mochila
  :pack-size 755
  ;; Los objetos - los inicializaré después de definir la función 
  ;;arrange-objects
  :objects objects
  ;; Tamaño inicial de la población
  :population-size 50
  ;; Número de individuos seleccionados por ronda en selección 
  ;; por torneo
  :tournament-size 5
  ;; Torneo con o sin reemplazamiento
  :replacement true
  ;; Probabilidad de activación de los genes cuando se genera un 
  ;; individuo
  :rand-gen-prob 5/10
  ;; Probabilidad utilizada para selección estocástica
  :stochastic-prob 8/10
  ;; Pobabilidad de mezcla en crossover
  :crossover-prob 5/10
  ;; Número de generaciones tope para el experimento
  :max-generations 200
  ;; Número de generaciones sin mejora del fitness que se permiten antes 
  ;; de dar por acabado el experimento
  :idle-generations 20
  ;; Cada cuantas generaciones se reporta el estado al web-service 
  ;; en la nube para su visualización
  :report-delta 1
  ;; Nombre del experimento, para poder monitorizarlo apropiadamente
  :name "El nombre que sea"}
```  

Por ejemplo, una ejecución del algoritmo de aprendizaje podría hacerse así:  


```clojure
;; Un conjunto de objetos de prueba
(def some-objects  (map #(rand-object % 20 20) (range 200)))

;; Plantilla para experimentos con objetos autogenerados
;; y configuración ad-hoc
(comment
  (->> {:pack-size 755
        :objects some-objects
        :population-size 50
        :tournament-size 5
        :replacement true
        :rand-gen-prob 5/10
        :stochastic-prob 8/10
        :crossover-prob 5/10
        :max-generations 200
        :idle-generations 20
        :report-delta 1
        :name "amanas: Autogenerado 1"}
       go-live
       decode))
```    



## Ejecución desde fichero del algoritmo

Sin embargo, en las indicaciones sobre como realizar la práctica se hace saber que **sería recomendable que se pudiera proceder a la ejecución de un experimento leyendo tantos los datos como la configuración desde un fichero**. Por lo tanto, también se permite proceder de este modo haciendo una llamada como la siguiente:

```clojure
;; Inicializa y lleva a cabo la evolución. Reporta el resultado.
;; Devuelve el mejor individuo encontrado.
;; - path: ruta al fichero con objetos y configuración a utilizar
;; El ficero tiene que tener un formato como el siguiente:
;; {:pack-size 1234
;;  :population-size 2
;;  :tournament-size 2
;;  :replacement true
;;  :rand-gen-prob 1/10
;;  :stochastic-prob 1/10
;;  :crossover-prob 1/10
;;  :max-generations 200
;;  :idle-generations 10
;;  :report-delta 1
;;  :name "amanas: Todo desde fichero 1"
;;  :objects [["objeto 1" 150 9]
;;            ["objeto 2" 120 8]
;;            ...]}
(defn go-live-from-file [path]...
```

Un ejemplo de fichero que cumpliría con los requisitos anteriores sería:

```clojure
{:pack-size 400
 :population-size 5
 :tournament-size 5
 :replacement true
 :rand-gen-prob 5/10
 :stochastic-prob 8/10
 :crossover-prob 5/10
 :max-generations 200
 :idle-generations 20
 :report-delta 1
 :name "amanas: Rosseta Code desde fichero"
 :objects [["map" 150 9]
           ["compass" 35 13]
           ["water" 200 153]
           ["sandwich" 160 50]
           ["glucose" 60 15]
           ...]}
```


Y con estas aclaraciones, creo que ya podemos pasar a comentar las experimentaciones que he realizado siguiendo el guión de la práctica.


# Evaluación Experimental

Tal como se indica en la práctica, he generado dos conjuntos de objetos, uno simple y otro complejo. He ido variando el número de individuos seleccionados en cada ronda de la fase de selección por torneo para así poder contrastar las implicaciones de tal parámetro en el rendimiento de la búsqueda de la solución.

Para cada tamaño de la selección por torneo, he ejecutado el algoritmo varias veces, de modo que pueda obtener medias de los rendimientos y poder así realizar un analísis más consistente.

Todas las ejecuciones que he realizado han quedado guardadas en el servicio que tengo publicado en la nube y que permite revisar tranquilamente tales ejecuciones. De todos modos, para evitar problemas de eliminado de caches, también he guardado los resultados en la carpeta "results" que se adjunta con este documento.

Por ejemplo, la Figura 2 muesrta el resultado de la primera ejecución del algoritmo para el conjunto simple con 2 individuos en la selección por torneo.

![Figura 2. Caso simple con tamaño de torneo 2](ai/ce/p1/doc/img/simple-2-0.png)

## Análisis de resultados

Con el siguiente script podemos agregar todos los resultados y así poder tomar medias. La forma más sencilla es descargarlos desde el servidor y procesarlos aquí. Para prevenir posibles accidentes (que se borre la caché del servidor, guardo una copia de los resultados en el proyecto).

In [117]:
import os.path
import json
import requests
import pandas as pd

def get_exp_name(type,tournament,run):
    template='amanas: %s - tournament %s - run %s'
    return template % (type,tournament,run)

def get_exp_url(type,tournament,run):
    template='http://amanas.ml/ce/service.php?action=get&name=%s'
    return template % get_exp_name(type,tournament,run)

def get_exp_path(type,tournament,run):
    template='/home/ubuntu/ai/ce/p1/results/%s.json' 
    return template % get_exp_name(type,tournament,run)

def get_exp_data(type,tournament,run):
    path=get_exp_path(type,tournament,run)
    if os.path.isfile(path):
        with open(path) as data:    
            return json.load(data)
    url=get_exp_url(type,tournament,run)
    data=requests.get(url).json()
    with open(path, 'w') as fp:
        json.dump(data, fp)
    return data

def to_DataFrame(type,tournaments,runs):
    data = []
    for tour in tournaments:
        for run in runs:
            d = get_exp_data(type,tour,run)
            data += [[type,tour,run,d['best-volume'],
                      d['best-fitness'],d['generation'],
                      (d['current-time']-d['start-time'])/1000]]
    df=pd.DataFrame(data)
    df.columns=['type','tournament','run','best-volume',
                'best-fitness','generation','seconds']
    return df

## Experimento simple

En el caso del experimento simple, he generado un fichero de datos que se puede consultar en "resources/data/simple.edn". El experimento se caracteriza por:

- 10 individuos
- 100 objetos
- con valor y volumen aleatoriamente entre [1, 100]
- y con capacidad de la mochila un valor aleatorio en el intervalo real [100, 10.000]

Para cada ejecución del algoritmo, he utilizado estos mismos datos pero he ido cambiando dos propiedades de la configuración (el nombre del experimento y el tamaño de la ronda por torneo). 

Así, finalmente he consiguido las siguientes ejecuciones, que se pueden consultar en [http://amanas.ml/ce/status.html](http://amanas.ml/ce/status.html)

Los resultados obtenidos son los siguientes:

In [121]:
simpleDF=to_DataFrame('simple',[2,3,4,5,6,10],range(10))
print(simpleDF)

      type  tournament  run  best-volume  best-fitness  generation  seconds
0   simple           2    0       501.42       1889.87          19    3.213
1   simple           2    1       509.49       1860.71           7    2.160
2   simple           2    2       497.08       1758.05           9    1.599
3   simple           2    3       505.08       1858.32          12    1.947
4   simple           2    4       510.19       1827.41          17    2.856
5   simple           2    5       510.22       1983.09          36    5.867
6   simple           2    6       513.15       1864.21          13    2.191
7   simple           2    7       507.78       1678.53           7    1.268
8   simple           2    8       492.92       1688.05          11    1.864
9   simple           2    9       498.98       1863.14          19    3.177
10  simple           3    0       516.76       1801.18          18    3.273
11  simple           3    1       512.43       1677.87           6    1.412
12  simple  

De dónde, agrupando por el tamaño del torneo, tenemos:    

In [122]:
grouped=simpleDF.groupby('tournament')

podemos observar los valores medios agrupados por tamaño del torneo:

In [123]:
print(grouped.mean())

            run  best-volume  best-fitness  generation  seconds
tournament                                                     
2           4.5      504.631      1827.138        15.0   2.6142
3           4.5      507.922      1799.655        14.1   2.4323
4           4.5      505.693      1865.845        17.2   2.8836
5           4.5      507.299      1782.554        14.2   2.3863
6           4.5      504.197      1803.437        14.4   2.4247
10          4.5      504.475      1824.345        13.8   2.3736


y las desviaciones medias agrupadas por tamaño del torneo:

In [124]:
print(grouped.std())

                run  best-volume  best-fitness  generation   seconds
tournament                                                          
2           3.02765     6.706961     93.825630    8.628119  1.314480
3           3.02765     6.338068    117.978915    5.762908  0.902846
4           3.02765     9.765144     82.929858    8.664102  1.373836
5           3.02765     9.745249     74.700340    6.941021  1.113167
6           3.02765     7.441466     96.715940    5.581716  0.888852
10          3.02765     9.859751    134.566208    7.509993  1.299723


### Conclusiones del experimento simple

En el caso simple no observo que el tamaño del torneo condicione el resultado del experimento.

Observamos que las medias del mejor fitness obtenido no siguen un patron de aumento o disminución en la medida que se incrementa el tamaño del torneo. Por ejemplo, con tamaño 2, la media del mejor fitness es 1827.138, que aumenta y disminuye según se aumenta el tamaño del torneo sin que parezca que sigue un patrón claro de comportamiento.

El tiempo medio de ejecución del algoritmo tampoco parece verse condicionado por el tamaño del torneo.

Igualmente, el número de generaciones necesarias para encontrar una solución que no se mejore en el transcurso de 1 generaciones parece mantenerse uniforme independientemente del tamaño del torneo.

Por lo tanto, al amparo de los datos experimentales arrojados por mis pruebas, me aventuro a concluir que en experimentos pequeños no hay gran diferencia entre **explotación** (tamaño de torneo pequeño) y **exploración** (tamaño de torneo mayor). 

Es decir, evolucionar utilizando la diversidad genética de individuos con inferior fitness no parece que aporte grandes mejoras a la solución encontrada en problemas de complejidad pequeña.


Evaluación del algoritmo genético

Conclusiones